In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from pandas.tseries import offsets

In [70]:
df = pd.read_csv('../input/nwdc-segregation-log.csv.gz', sep='|', quotechar='"', compression='gzip')

In [71]:
df['start_dt'] = pd.to_datetime(df['Placement Date'])
df['end_dt'] = pd.to_datetime(df['Release Date'])

In [72]:
assert sum(df['start_dt'].isnull()) == 0

In [73]:
sum(df['end_dt'].isnull())

14

In [74]:
df['solitary_length'] = df['end_dt'] - df['start_dt']

In [75]:
df['solitary_length'].describe()

count                        357
mean     59 days 14:31:15.630252
std      88 days 06:18:57.166750
min              1 days 00:00:00
25%             19 days 00:00:00
50%             30 days 00:00:00
75%             62 days 00:00:00
max            781 days 00:00:00
Name: solitary_length, dtype: object

In [76]:
df['solitary_days'] = df['solitary_length'] / np.timedelta64(1, 'D')

In [77]:
df['solitary_days'].describe()

count    357.000000
mean      59.605042
std       88.263162
min        1.000000
25%       19.000000
50%       30.000000
75%       62.000000
max      781.000000
Name: solitary_days, dtype: float64

In [78]:
df['solitary_days'].max()/30

26.033333333333335

In [79]:
df.columns

Index(['Tracking Number', 'Alien Number', 'Last Name', 'First Name', 'Gender',
       'Date of Birth', 'Country of Citizenship', 'Facility AOR', 'Facility',
       'Facility:DETLOC', 'Report Type', 'Placement Date', 'Placement Reason',
       'Release Date', 'Length of Stay', 'Disciplinary Infraction',
       'Sanction Length', 'Detailed Reason', 'Attorney of Record',
       'Attorney Notification', 'Detainee Request',
       'Compliance with Detention Standards', 'Mental Illness',
       'Serious Medical Illness', 'Serious Disability', 'Ever SMI?',
       'FO Recommendation', 'Additional Comments Export',
       'Non Compliance Detail', 'Special Criteria', 'ID', 'Current Review',
       'Created', 'LGBTI', 'Item Type', 'Path', 'start_dt', 'end_dt',
       'solitary_length', 'solitary_days'],
      dtype='object')

**Category 5.1** Any inmate who is is “held continuously in segregation for 14 days, 30 days, and at every 30-day interval thereafter, or has been held in segregation for 14 days out of any 21 day period.”)


**Category 5.2.2.** Any inmate who, immediately after being assigned to solitary, is determined to fall into one of these particularly vulnerable categories: has a disability, medical or mental illness, or other special vulnerability; or who is an alleged victim of a sexual assault, an identified suicide risk, or is on a hunger strike.

In [80]:
fod1 = df['solitary_days'] >= 14
fod2 = df['solitary_days'] >= 30

In [81]:
df['FOD1'] = df[fod1]['start_dt'] + datetime.timedelta(days=14)

In [82]:
df['FOD2'] = df[fod2]['start_dt'] + datetime.timedelta(days=30)

In [83]:
interval = 30

In [84]:
df[df['solitary_days'] >= 60]['solitary_days'].head()

22     83.0
27     91.0
29     66.0
35     77.0
38    101.0
Name: solitary_days, dtype: float64

In [85]:
df['additional_FOD_intervals'] = (df[df['solitary_days'] >= 60]['solitary_days'] - 30) // 30

In [86]:
df['total_FOD'] = df['additional_FOD_intervals'] + 2

In [101]:
df['total_FOD'] = df['total_FOD'].fillna(0)

In [87]:
df['Placement Reason'].unique()

array(['Medical: Observation',
       'Protective Custody: Other Detainee Safety',
       'Facility Security Threat: Other', 'Disciplinary',
       'Pending Investigation of Disciplinary Violation',
       'Facility Security Threat: Violent or Disruptive Behavior',
       'Protective Custody: Gang Status (Protective Custody Only)',
       'Protective Custody: Special Vulnerability Other',
       'Protective Custody: Lesbian, Gay, Bisexual, Transgender (LGBT)',
       'Facility Security Threat: Gang Member Status (Not Protective Custody)',
       'Protective Custody: Criminal Offense (i.e. Sex Offender)',
       'Mental Illness', 'Other', 'Medical: Other',
       'Facility Security Threat: Due to Seriousness of Criminal Conviction',
       'Hunger Strike'], dtype=object)

In [95]:
vulnerable_cat = ['Medical: Observation',
              'Protective Custody: Other Detainee Safety',
              'Protective Custody: Special Vulnerability Other',
              'Protective Custody: Lesbian, Gay, Bisexual, Transgender (LGBT)',
              'Mental Illness', 'Medical: Other',
              'Hunger Strike']

In [96]:
vulnerable = df['Placement Reason'].isin(vulnerable_cat)

In [97]:
df['vulnerable_initial_FOD'] = df[vulnerable]['start_dt']

In [106]:
df.loc[vulnerable, 'total_FOD'] = df.loc[vulnerable, 'total_FOD'] + 1

In [107]:
df[vulnerable]['total_FOD']

0       1.0
1       1.0
5       1.0
8       1.0
13      1.0
15      1.0
21      1.0
22      4.0
23      1.0
25      1.0
29      4.0
34      1.0
36      1.0
44      5.0
47      1.0
49      1.0
53      1.0
54      1.0
55      7.0
56      1.0
59      4.0
60      1.0
61      1.0
62      1.0
66      6.0
67      8.0
68      1.0
69      1.0
72      5.0
74      1.0
       ... 
198     7.0
199     1.0
200     1.0
202     1.0
215     1.0
225     6.0
255     1.0
259    11.0
264     6.0
280     1.0
282     1.0
284     1.0
286     1.0
288     1.0
290     1.0
296     1.0
298     1.0
303     1.0
305     1.0
306     4.0
326     1.0
333     1.0
334     4.0
335     1.0
336     1.0
337     1.0
361     1.0
363     4.0
365     1.0
366     1.0
Name: total_FOD, Length: 114, dtype: float64

In [108]:
df['total_FOD'].max()

27.0

In [110]:
sum(df['total_FOD'] >= 1)

163

In [112]:
len(df[vulnerable])

114

In [114]:
sum(df['total_FOD'])

657.0

In [127]:
print(list(df[df['total_FOD'] >= 1]['Tracking Number']))

[12442, 12439, 12342, 12240, 12112, 12110, 11944, 11943, 11594, 11592, 11587, 11456, 11353, 11263, 11206, 11133, 11112, 11111, 11038, 11035, 11017, 10691, 10690, 10688, 10635, 10618, 10582, 10431, 10430, 10429, 10125, 10122, 10045, 10042, 9769, 9679, 9678, 9677, 9676, 9675, 9488, 9487, 9459, 9441, 9439, 9438, 9423, 9394, 9374, 9361, 9342, 9295, 9230, 9222, 9180, 9111, 9107, 9015, 8500, 8461, 8460, 8447, 8436, 8410, 8346, 8196, 8193, 7636, 7523, 7305, 7198, 7059, 6659, 6373, 6372, 6371, 6336, 6222, 6094, 6093, 6055, 6007, 5968, 5959, 5741, 5740, 5627, 5604, 5535, 5528, 5516, 5515, 5308, 5132, 5131, 5130, 5075, 4887, 4762, 4738, 4735, 4587, 4511, 4484, 4483, 4466, 4450, 4449, 4448, 4446, 4445, 4441, 4418, 4403, 3808, 3511, 3504, 3503, 3466, 3334, 3286, 3087, 3076, 2938, 2871, 2870, 2699, 2573, 2372, 2330, 2085, 1727, 1726, 1676, 1662, 1661, 1652, 1617, 1594, 1503, 1469, 1427, 1386, 1303, 1234, 965, 860, 798, 750, 749, 700, 699, 694, 399, 328, 307, 281, 264, 262, 259, 252, 233, 210]


In [122]:
df.tail()

,Tracking Number,Alien Number,Last Name,First Name,Gender,Date of Birth,Country of Citizenship,Facility AOR,Facility,Facility:DETLOC,...,Path,start_dt,end_dt,solitary_length,solitary_days,FOD1,FOD2,additional_FOD_intervals,total_FOD,vulnerable_initial_FOD
366,259,"(b)(6), (b)(7)(c )","(b)(6), (b)(7)(c )","(b)(6), (b)(7)(c )",Male,"(b)(6), (b)(7)(c )",RUSSIA,SEA (Seattle),TACOMA ICE PROCESSING CENTER (NORTHWEST DET CT...,CSCNWWA,...,(b)(7)(E),2013-06-24,2013-07-30,36 days,36.0,2013-07-08,2013-07-24,NaN,1.0,2013-06-24
367,252,"(b)(6), (b)(7)(c )","(b)(6), (b)(7)(c )","(b)(6), (b)(7)(c )",Male,"(b)(6), (b)(7)(c )",MEXICO,SEA (Seattle),TACOMA ICE PROCESSING CENTER (NORTHWEST DET CT...,CSCNWWA,...,(b)(7)(E),2013-06-19,2014-11-10,509 days,509.0,2013-07-03,2013-07-19,15.0,17.0,NaT
368,233,"(b)(6), (b)(7)(c )","(b)(6), (b)(7)(c )","(b)(6), (b)(7)(c )",Male,"(b)(6), (b)(7)(c )",PALAU,SEA (Seattle),TACOMA ICE PROCESSING CENTER (NORTHWEST DET CT...,CSCNWWA,...,(b)(7)(E),2013-06-03,2013-08-15,73 days,73.0,2013-06-17,2013-07-03,1.0,3.0,NaT
369,219,"(b)(6), (b)(7)(c )","(b)(6), (b)(7)(c )","(b)(6), (b)(7)(c )",Male,"(b)(6), (b)(7)(c )",BOSNIA-HERZEGOVINA,SEA (Seattle),TACOMA ICE PROCESSING CENTER (NORTHWEST DET CT...,CSCNWWA,...,(b)(7)(E),2013-05-24,2013-07-16,53 days,53.0,2013-06-07,2013-06-23,NaN,0.0,NaT
370,210,"(b)(6), (b)(7)(c )","(b)(6), (b)(7)(c )","(b)(6), (b)(7)(c )",Male,"(b)(6), (b)(7)(c )",MEXICO,SEA (Seattle),TACOMA ICE PROCESSING CENTER (NORTHWEST DET CT...,CSCNWWA,...,(b)(7)(E),2013-05-13,2013-08-15,94 days,94.0,2013-05-27,2013-06-12,2.0,4.0,NaT


In [126]:
df.loc[df['solitary_days'].idxmax()]['Placement Reason']

'Protective Custody: Criminal Offense (i.e. Sex Offender)'